In [5]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import ndlib.models.epidemics as ep
import ndlib.models.ModelConfig as mc
from aesara import tensor as at
from bokeh.io import output_notebook, show
# from ndlib.viz.bokeh.DiffusionTrend import DiffusionTrend
from ndlib.viz.mpl.TrendComparison import DiffusionTrendComparison
import pysindy as ps


WARNING (aesara.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (aesara.configdefaults): g++ not detected!  Aesara will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set Aesara flags cxx to an empty string.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


no display found. Using non-interactive Agg backend


In [6]:
def get_trajectory_matrix(trend):
    return np.array([trend[0]['trends']['node_count'][i] for i in range(3)])

In [7]:
# Network topology
N_pop = 200
g = nx.erdos_renyi_graph(N_pop, .3)
N_sim = 100
# 2° Model selection
models = [ep.SIRModel(g) for i in range(N_sim)]

# 2° Model Configuration
cfg = mc.Configuration()
cfg.add_model_parameter('beta', 0.01)
cfg.add_model_parameter('gamma', 0.02)
cfg.add_model_parameter("fraction_infected", 0.1)
[model.set_initial_status(cfg) for model in models]
Nt = 200
trends = []
X = np.zeros((N_sim, 3, Nt))
for i, model in enumerate(models):
    iteration = model.iteration_bunch(Nt)
    
    trends.append(model.build_trends(iteration))
    X[i, :, :] = get_trajectory_matrix(trends[-1])
# 2° Simulation execution




In [9]:
G = models[0].graph.graph
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
import matplotlib
matplotlib.use('QtAgg')
nx.draw(G, width=.1, style='dashed', edge_color='r', node_color='k', ax=ax)
fig.savefig('network.eps', format='eps')

In [10]:
fig, ax = plt.subplots(3)
for traj in X:
    ax[0].plot(traj[0,:], color='y')
    ax[1].plot(traj[1,:], color='r')
    ax[2].plot(traj[2,:], color='b')
plt.show()

In [11]:
t = [*list(range(Nt))]*N_sim

In [12]:
from Quantile_FROLS import Quantile_FROLS
X_2D = np.hstack([xk for xk in X])
threshold = 1e-6
reg_model = ps.SINDy(Quantile_FROLS(verbose=True),
feature_library=ps.PolynomialLibrary(degree=2))


reg_model.fit(X_2D[:,:Nt].T)
# dmd = DMD()
# dmd.fit(X_2D)
# for dynamics in dmd.dynamics:
#     plt.plot(t, dynamics.real)
reg_model.print()


AttributeError: module 'pysindy' has no attribute 'Quantile_FROLS'

In [ ]:
import matplotlib
plt.ioff()
matplotlib.use('QtAgg')
sim = reg_model.simulate(x0=X_2D[:,Nt], t=np.linspace(0,200, 10000))
plt.plot(sim)
plt.show()

NameError: name 'plt' is not defined